In [15]:
import os
from http.cookiejar import MozillaCookieJar

import cloudscraper
from bs4 import BeautifulSoup

from importlib import reload

import src.utility.url_manager as url
import src.website.shoot as sh
import src.files.video as vd

reload(url)
reload(sh)
reload(vd)

<module 'src.files.video' from '/media/gmagostini/Elements/kink-downloader/src/files/video.py'>

In [2]:
cookie_file = os.path.expanduser("~/cookies.txt")
base_url = 'www.kink.com'
session = cloudscraper.CloudScraper()
cookie = MozillaCookieJar(cookie_file)
cookie.load(ignore_expires=False, ignore_discard=False)

#### gallery

In [3]:
site = url.UrlManager(url.UrlManager.HttpType.HTTPS, base_url, ['my', 'favorite-scenes', 'page', '2'])
site.get_url()

'https://www.kink.com/my/favorite-scenes/page/2/'

In [4]:
req = session.get(site.get_url(), cookies=cookie)
soup = BeautifulSoup(req.text, "html.parser")

number of page in favorite shoot

In [5]:
last_page_of_favorites = soup.select('li.page-item:last-child > a:nth-child(1)')[0].get_text()
int(last_page_of_favorites)

92

In [6]:
with open("output.html", "w") as file:
    file.write(str(soup.prettify()))

In [7]:
path = os.path.abspath(os.path.join(os.path.abspath(''), '..', 'kink'))
element = soup.find_all('a', 'd-block')[0]

In [16]:
first = sh.Shoot(href=element.get('href'), cookie=cookie, path=path)

shoot: <Shoot: 00034226:__________________Katherine Kane: Champion Anal Electroslut Live Show!>


In [17]:
first.get_data_for_datatable()

{'number': '34226',
 'title': 'Katherine Kane: Champion Anal Electroslut Live Show!',
 'description': None,
 'quality': '720',
 'video': 'https://cdn.kink.com/electrosluts/members/34226/video/h264/full/34226_Katharine_Aiden.mp4?nva=1701454915&token=3699e8fd3ab9814ef655aabc5556de12',
 'poster': <Poster: 34226 - 720p.jpg>,
 'zip_image': <ZipFile: 34226.zip>}

In [ ]:
shots = [sh.Shoot(href=el.get('href'), cookie=cookie, path=path) for el in soup.select('div.col-sm-6 > div:nth-child(1) > div:nth-child(2) > a')]

In [ ]:
shots[0].title

In [ ]:
len(soup.select('div.col-sm-6 > div:nth-child(1) > div:nth-child(2) > a') )

In [ ]:
req = session.get(first.get_url(), cookies=cookie)
soup = BeautifulSoup(req.text, "html.parser")

In [ ]:
soup.find_all('h1')[0].get_text() == '404'

In [ ]:
soup.find('h1', 'shoot-title').get_text()

In [ ]:
videos = [vd.Video(val.get('download'), first.number, val.get('quality'), path) for val in
          soup.find_all('a', download=True, quality=True)]
videos

In [ ]:
videos[0].download()

In [ ]:
first.poster

In [ ]:
zip_image = soup.find('a', 'zip-links', download=True).get('download')
zip_image

In [ ]:
actors = soup.select('.names > a')
actors